<a href="https://colab.research.google.com/github/sourcecode369/TensorFlow-2.0/blob/master/tensorflow_2.0_docs/TensorFlow%20Core/Tutorials/Distributed%20Training/Multi-worker%20Training%20with%20Keras/TensorFlow_2_0_Distributed_Training_Multi_worker_training_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade tensorflow-gpu

     |████████████████████████████████| 380.8MB 43kB/s 
     |████████████████████████████████| 3.8MB 28.7MB/s 
     |████████████████████████████████| 450kB 50.6MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.6.3 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existi

In [0]:
from __future__ import absolute_import, print_function, division, unicode_literals
import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

In [2]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def make_datasets_unbatched():
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

  datasets, info = tfds.load(name='mnist', 
                             with_info=True,
                             as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE)

train_datasets = make_datasets_unbatched().batch(BATCH_SIZE)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/1.0.0. Subsequent calls will reuse this data.


In [0]:
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
                               tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, input_shape=(28,28,1)),
                               tf.keras.layers.MaxPooling2D(),
                               tf.keras.layers.Flatten(),
                               tf.keras.layers.Dense(64,activation=tf.nn.relu),
                               tf.keras.layers.Dense(10, activation="softmax")
  ])
  model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics = ["accuracy"]
                )
  return model

In [4]:
single_worker_model = build_and_compile_cnn_model()
history = single_worker_model.fit(train_datasets, 
                                  epochs=3)

Epoch 1/3
938/938 [==============================] - 24s 25ms/step - loss: 2.0215 - accuracy: 0.4365
Epoch 2/3
938/938 [==============================] - 5s 5ms/step - loss: 1.1274 - accuracy: 0.7554
Epoch 3/3
938/938 [==============================] - 5s 5ms/step - loss: 0.6088 - accuracy: 0.8595


In [0]:
# import os
# import json
# os.environ["TF_CONFIG"] = json.dumps(
#     {
#         'cluster': {
#             'worker': ["localhost:12345", "localhost:23456"]
#         },
#         'task': {
#             'type':'worker', 'index':0
#         }
#     }
# )

### Choosing the right strategy

In [6]:
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(communication=tf.distribute.experimental.CollectiveCommunication.RING)

INFO:tensorflow:Single-worker CollectiveAllReduceStrategy with local_devices = ('/device:GPU:0',), communication = CollectiveCommunication.RING


INFO:tensorflow:Single-worker CollectiveAllReduceStrategy with local_devices = ('/device:GPU:0',), communication = CollectiveCommunication.RING


In [8]:
print(f"Number of parallel devices: {strategy.num_replicas_in_sync}")

Number of parallel devices: 1


In [10]:
%%time
NUM_WORKERS = 2
GLOBAL_BATCH_SIZE = 64 * NUM_WORKERS
with strategy.scope():
  train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)
  multi_worker_model = build_and_compile_cnn_model()
multi_worker_model.fit(train_dataset,epochs=3)

Epoch 1/3
469/469 [==============================] - 20s 43ms/step - loss: 2.2002 - accuracy: 0.3626
Epoch 2/3
469/469 [==============================] - 3s 7ms/step - loss: 1.9116 - accuracy: 0.6093
Epoch 3/3
469/469 [==============================] - 3s 7ms/step - loss: 1.4123 - accuracy: 0.7465
CPU times: user 32.5 s, sys: 6.52 s, total: 39 s
Wall time: 27.9 s


### Dataset sharding and batch size

In [0]:
options = tf.data.Options()
options.experimental_distribute.auto_shard = False
train_datasets_no_auto_shard = train_datasets.with_options(options)

### Performance

##### 1. Use different collective communication implementations. 

    tf.distribute.experimental.CollectiveCommunication.NCCL
    tf.distribute.experimental.CollectiveCommunication.RING
    tf.distribute.experimental.CollectiveCommunication.AUTO

##### 2. Cast the variables to `tf.float`

### Fault Tolerance

In synchronous training, the cluster would fail if one of the workers fail and no fault tolerance mechanism exists.
Using tf.distribute.Strategy comes with the advantage of fault tolerance i.e. where workers die or are otherwise unstable.

This is done by preserving the training state in the distributed file system such that upon restart of the instance that previously failed or preempted, the training is recovererd.

### Model Checkpoint Callback

In [14]:
%%time
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath='/tmp/keras-ckpt/')]

with strategy.scope():
  option = tf.data.Options()
  option.experimental_distribute.auto_shard = True
  dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)
  dataset_with_shard = dataset.with_options(option)
  multiworker_model = build_and_compile_cnn_model()
  multiworker_model.fit(dataset_with_shard,epochs=3,callbacks=callbacks)

Epoch 1/3
    469/Unknown - 19s 41ms/step - loss: 2.2514 - accuracy: 0.2505WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: /tmp/keras-ckpt/assets


INFO:tensorflow:Assets written to: /tmp/keras-ckpt/assets


469/469 [==============================] - 20s 42ms/step - loss: 2.2514 - accuracy: 0.2505
Epoch 2/3
468/469 [============================>.] - ETA: 0s - loss: 2.0501 - accuracy: 0.5217INFO:tensorflow:Assets written to: /tmp/keras-ckpt/assets


INFO:tensorflow:Assets written to: /tmp/keras-ckpt/assets


469/469 [==============================] - 4s 9ms/step - loss: 2.0501 - accuracy: 0.5220
Epoch 3/3
467/469 [============================>.] - ETA: 0s - loss: 1.6304 - accuracy: 0.7087INFO:tensorflow:Assets written to: /tmp/keras-ckpt/assets


INFO:tensorflow:Assets written to: /tmp/keras-ckpt/assets


469/469 [==============================] - 4s 8ms/step - loss: 1.6296 - accuracy: 0.7091
CPU times: user 32.3 s, sys: 6.38 s, total: 38.7 s
Wall time: 29 s
